In [1]:
from typing import Callable

from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
import torch

from utils.load_data import *
import numpy as np
from scipy import signal

BCIC_dataset = load_BCIC(
train_sub=[1,2,3,4,5,6,7,8],
test_sub=[9],
alg_name = 'Tensor_CSPNet',
scenario = 'raw-signal-si'
)

train_x, train_y, test_x, test_y = BCIC_dataset.generate_training_valid_test_set_subject_independent()

class BCI_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float32), self.y[idx]
    
train_dataset = BCI_dataset(x=train_x, y=train_y)
test_dataset = BCI_dataset(x=test_x, y=test_y)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=16)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=16)

/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-04-26 17:14:53.029314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 17:14:53.582845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch
from diffusers import UNet2DModel
from consistency_models import consistency
from consistency_models.loss import PerceptualLoss

consistency = consistency.Consistency(
    model=UNet2DModel(sample_size=22),
    loss_fn=PerceptualLoss(net_type=("squeeze"))
).cuda()

/home/junyeobe/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/junyeobe/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/junyeobe/augment/consistency_models/consistency.py:57: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DModel's config object instead, e.g. 'unet.config.in_channels'.
  self.channels = model.in_channels


In [3]:
from pytorch_lightning import Trainer

trainer = Trainer(max_epochs=8000, accelerator="auto")
trainer.fit(consistency, train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type           | Params
------------------------------------------------------
0 | model              | UNet2DModel    | 274 M 
1 | model_ema          | UNet2DModel    | 274 M 
2 | loss_fn            | PerceptualLoss | 724 K 
3 | _loss_tracker      | MeanMetric     | 0     
4 | _bins_tracker      | MeanMetric     | 0     
5 | _ema_decay_tracker | MeanMetric     | 0     
-------------------------------------------

Training: 0it [00:00, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_20488/814284149.py:4 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_20488/814284149.py'                         │
│                                                                                                  │
│ /home/junyeobe/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:520 in    │
│ fit                                                                                              │
│                                                                                                  │
│    517 │   │   """                                                                               │
│    518 │   │   model = _maybe_unwrap_optimized(model)                                            │
│    519 │   │   self.strategy._lightning_module = model                                           │
│ ❱  520 │   │   call._call_and_handle_interrupt(                                                  │
│    521 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    522 │   │   )                                                                                 │
│    523                                                                                           │
│                                                                                                  │
│ /home/junyeobe/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:44 in        │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│    41 │   │   if trainer.strategy.launcher is not None:                                          │
│    42 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,    │
│    43 │   │   else:                                                                              │
│ ❱  44 │   │   │   return trainer_fn(*args, **kwargs)                                             │
│    45 │                                                                                          │
│    46 │   except _TunerExitException:                                                            │
│    47 │   │   _call_teardown_hook(trainer)                                                       │
│                                                                                                  │
│ /home/junyeobe/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:559 in    │
│ _fit_impl                                                                                        │
│                                                                                                  │
│    556 │   │   │   model_provided=True,                                                          │
│    557 │   │   │   model_connected=self.lightning_module is not None,                            │
│    558 │   │   )                                                                                 │
│ ❱  559 │   │   self._run(model, ckpt_path=ckpt_path)                                             │
│    560 │   │                                                                                     │
│    561 │   │   assert self.state.stopped                                                         │
│    562 │   │   self.training = False                                                             │
│                                                                                                  │
│ /home/junyeobe/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:935 in    │
│ _run                                                       